In [2]:
##======================================[ zone d'import ]========================================================================
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio 
import os

##======================================[ zone des constantes ]==================================================================

##======================================[ zone du main ]========================================================================
if __name__ =="__main__" :
    plt.close("all")

##--------------------------------------[ récupération dataset ]---------------------------------------------------
def get_files_with_extension(folder_path, extension):               # Récupère les éléments d'un dossier
    """
    Récupère tous les fichiers avec une extension spécifique dans un dossier.

    :param folder_path: Chemin vers le dossier
    :param extension: Extension de fichier (par exemple, '.txt')
    :return: Liste des chemins des fichiers ayant l'extension spécifiée
    """
    files = [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if file.endswith(extension)
    ]
    return files

def get_files_with_extension_recursive(folder_path, extension):     # Récupère les éléments dans tout les dossier présent dans un dossier
    """
    Récupère tous les fichiers avec une extension spécifique dans un dossier et ses sous-dossiers.

    :param folder_path: Chemin vers le dossier
    :param extension: Extension de fichier (par exemple, '.txt')
    :return: Liste des chemins des fichiers ayant l'extension spécifiée
    """
    files = []
    # Parcourt tous les fichiers et dossiers récursivement
    for root, dirs, filenames in os.walk(folder_path):
        for file in filenames:
            if file.endswith(extension):
                files.append(os.path.join(root, file))
    return files
    
##--------------------------------------[ Fonction de calcul ]-----------------------------------------------------
def uniformity_calcul(Img, nbr_case, nb_lg, nb_col):                # Découpe l'image et Calcul l'uniformité des sous-images
    """
    Calculer l'uniformité des blocs d'image.

    :param Img: Image d'entrée (tableau numpy)
    :param nbr_case: Nombre de blocs (blocs par ligne/colonne)
    :param nb_lg: Nombre de lignes dans l'image
    :param nb_col: Nombre de colonnes dans l'image
    :return: Tab_IMG (blocs d'image) et matrice d'uniformité
    """

    # Determine the number of channels in the image
    if len(Img.shape) == 2:
        # Grayscale image, add a channel dimension
        Img = np.expand_dims(Img, axis=-1)
    
    nb_channel = Img.shape[2]  # Number of channels
    
    # Initialize the block array
    Tab_IMG = np.zeros(
        (nbr_case * nbr_case, nb_lg // nbr_case, nb_col // nbr_case, nb_channel),
        dtype=np.float32
    )
    uniformity = np.zeros((nbr_case, nbr_case), dtype=np.float32)  # Uniformity matrix

    # Divide the image into blocks
    indice = 0
    for i in range(nbr_case):  # Rows of blocks
        for j in range(nbr_case):  # Columns of blocks
            # Extract the block
            block = Img[
                i * (nb_lg // nbr_case): (i + 1) * (nb_lg // nbr_case),
                j * (nb_col // nbr_case): (j + 1) * (nb_col // nbr_case),
                :
            ]
            Tab_IMG[indice] = block
            
            # Calculate the standard deviation of the block
            std_dev = np.std(block)
            
            # Calculate uniformity: higher uniformity = lower standard deviation
            uniformity[i, j] = max(0, 100 - (std_dev / np.max(block)) * 100)  # Normalize to percentage
            
            indice += 1

    return Tab_IMG, uniformity

def L1 (x,y): # Calcul de la distance entre 2 sous-images ( Distance L1 ) 
    """
    Calcul la distance de manhattan en les vecteurs x et y tel que :
         D = somme( |Xi - Yi| )

    Paramètres :
        X : Premier vecteur                         /    type = np.array of np.float
        Y : Deuxième vecteur                        /    type = np.array of np.flaot
    Returns :
        D : Distance entre les vecteurs x et y      /    type = float 
    """

    return np.sum(np.abs( x - y))

##--------------------------------------[ Fonction de Traitement ]-------------------------------------------------
# Fonction pour regrouper les distances dans 10 classes
def group_by_distance_class(matrix, num_classes=10):                # Fonction pour regrouper les distances dans 10 classes
    """
    Regroupe les distances dans 'num_classes' classes égales
    """
    # Normalisation des distances
    min_distance = np.min(matrix)
    max_distance = np.max(matrix)
    
    # Gestion des cas où max_distance = min_distance (éviter division par zéro)
    if max_distance == min_distance:
        step = 1  # Utilise un pas de 1 dans ce cas particulier
    else:
        step = (max_distance - min_distance) / num_classes

    # Création d'une nouvelle matrice de classes
    class_matrix = np.zeros_like(matrix, dtype=np.int32)

    # Classification des distances
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            # Si la distance est infinie, on la classe dans la dernière classe (la plus grande)
            if matrix[i, j] == np.inf:
                class_matrix[i, j] = num_classes - 1
            else:
                # Classement par plage
                class_matrix[i, j] = int((matrix[i, j] - min_distance) // step)
    
    return class_matrix

def check_uniform_background(class_matrix, num_classes=10):         # Fonction d'analyse ( Fond Uni OUI/NON )
    """
    Vérifie si la classe 1 est majoritaire pour déterminer si l'image a un fond uni ou non.

    Paramètres :
        class_matrix : Matrice des classes regroupées
        num_classes : Nombre de classes définies (par défaut 10)
    
    Retourne :
        0 ou 1 : Booléen indiquant si l'image a un fond uni ou non
    """
    count_class_0 = np.sum(class_matrix == 0)                       # Compter combien de blocs appartiennent à la classe 0
    if count_class_0 > (class_matrix.size // 3):                    # Critére de choix ( + 33 % d'unicité = Fond Uni )
        return 1
    else:
        return 0

##--------------------------------------[ Fonction d'affichage ]-------------------------------------------------
def affiche_matrice(mat, titre, label, cmap):                       # Affiche la matrice choisie
    """
    Affiche une matrice avec une taille de figure ajustée à ses dimensions.

    :param mat: La matrice à afficher (de type numpy.array ou similaire)
    :param titre: Titre de l'affichage
    :param label: Label de la barre de couleur
    :param cmap: Colormap pour l'affichage
    """
    rows, cols = mat.shape                                          # Détermine les dimensions de la matrice
    figsize = (cols / 2, rows / 2)                                  # Ajuste la taille de la figure en fonction des dimensions de la matrice et Divise les dimensions par 2 pour que l'affichage soit proportionné et agréable
    plt.figure(figsize=figsize)                                     # Crée une nouvelle figure avec la taille ajustée
    plt.imshow(mat, cmap=cmap, interpolation='nearest')             # Affiche la matrice avec la colormap et sans interpolation
    plt.colorbar(label=label)                                       # Ajoute une barre de couleur avec le label fourni
    plt.title(titre)                                                # Ajoute le titre de l'affichage
    plt.axis('off')                                                 # Supprime les axes pour un affichage plus clair

def affiche_block(Tab_IMG, nb_case):                                # Affiche l'image découpé 
    """
    Affiche une grille de blocs d'image.

    :param Tab_IMG: Liste ou tableau contenant les blocs d'image
    :param nb_case: Nombre de blocs par ligne et par colonne (grille de nb_case x nb_case)
    """
    # Crée une grille de sous-figures (axes) avec nb_case x nb_case blocs
    # figsize=(8,8) définit une taille fixe pour l'ensemble de la figure
    fig, axes = plt.subplots(nb_case, nb_case, figsize=(8, 8))
    
    # Parcourt tous les blocs d'image (nb_case * nb_case blocs au total)
    for k in range(nb_case * nb_case):
        # Calcule la position du bloc dans la grille
        row, col = k // nb_case, k % nb_case        # Ligne et colonne correspondantes
        ax = axes[row, col]                         # Récupère l'axe correspondant
        ax.imshow(Tab_IMG[k], cmap='gray')          # Affiche l'image du bloc dans l'axe avec une colormap en niveaux de gris
        ax.axis('off')                              # Supprime les axes pour une meilleure lisibilité
        ax.set_title(f"Bloc {k + 1}", fontsize=2)   # Ajoute un titre au-dessus de chaque bloc, indiquant son numéro et Taille de police réduite (fontsize=2) pour éviter le chevauchement
    plt.tight_layout()                              # Ajuste automatiquement l'espacement entre les blocs pour éviter les chevauchements

##--------------------------------------[ Fonction de test complète ]-------------------------------------------------

def test_image(lepng, nb_case, num_image, print_test):              # Testeur d'image sans affichage des matrices              
    """
    Détecte si une image a un fond uniforme.

    :param lepng: Chemin vers l'image à analyser
    :param nb_case: Nombre de blocs (par ligne et colonne)
    :param num_image: Numéro de l'image (pour l'affichage des résultats)
    :param print_test: Indicateur pour afficher ou non le résultat (1 pour afficher)
    :return: Statut indiquant si le fond est uniforme (1) ou non (0)
    """
    
    # Lecture de l'image
    Img = iio.imread(lepng)
    
    # Vérifie la structure de l'image (dimensions)
    if len(Img.shape) == 2:
        # Si l'image est en niveaux de gris, ajoute une troisième dimension (canaux)
        Img = np.expand_dims(Img, axis=-1)
    
    # Récupère le nombre de lignes, colonnes et canaux de l'image
    nb_lg, nb_col, nb_channel = Img.shape

    # Calcul de l'uniformité des blocs
    Tab_IMG, uniformity = uniformity_calcul(Img, nb_case, nb_lg, nb_col)

    # Calcul des distances (L1)
    max_uniformity_value = np.max(uniformity)  # Trouve la valeur maximale d'uniformité
    distances = np.zeros_like(uniformity, dtype=np.float32)  # Initialise une matrice de distances 

    # Parcourt chaque bloc pour calculer la distance L1 par rapport à la valeur maximale d'uniformité
    for i in range(nb_case):
        for j in range(nb_case):
            # Calcule la distance L1 entre l'uniformité actuelle et la valeur maximale
            distance = L1(np.array([uniformity[i, j]]), np.array([max_uniformity_value]))
            distances[i, j] = distance

    # Remplace les valeurs NaN dans la matrice des distances par une valeur infinie
    distances = np.nan_to_num(distances, nan=np.inf)

    # Regroupe les distances en classes (par exemple, 10 classes)
    grouped_distances = group_by_distance_class(distances, num_classes=10)

    # Vérifie si le fond est uniforme en analysant les distances regroupées
    background_status = check_uniform_background(grouped_distances)

    # Affiche le résultat si demandé
    if print_test == 1:
        if background_status == 1:
            print(f"Image {num_image} : Fond Uni (studio)" )
        else:
            print(f"Image {num_image} : Pas Fond Uni (report)" )
    
    # Retourne le statut du fond (1 pour uniforme, 0 sinon)
    return background_status

def test_image_affiche(lepng, nb_case, num_image, print_test):      # Testeur d'image avec affichage des matrices
    """
    Teste si une image a un fond uniforme avec affichage des matrices intermédiaires.

    :param lepng: Chemin de l'image à analyser
    :param nb_case: Nombre de blocs par ligne et par colonne
    :param num_image: Numéro de l'image (pour affichage des résultats)
    :param print_test: Indicateur pour afficher ou non le résultat (1 pour afficher)
    :return: Statut indiquant si le fond est uniforme (1) ou non (0)
    """
    
    # Lecture de l'image
    Img = iio.imread(lepng)
    
    # Récupération des dimensions de l'image (lignes, colonnes, canaux)
    nb_lg, nb_col, nb_channel = Img.shape

    # Calcul de l'uniformité des blocs
    Tab_IMG, uniformity = uniformity_calcul(Img, nb_case, nb_lg, nb_col)

    # Affichage de la matrice d'uniformité
    affiche_matrice(
        uniformity,
        "Matrice d'uniformité des blocs",
        "Uniformité (%)",
        "seismic_r"
    )

    # Calcul de la distance par rapport au bloc avec l'uniformité maximale
    max_uniformity_value = np.max(uniformity)  # Trouve l'uniformité maximale
    distances = np.zeros_like(uniformity, dtype=np.float32)  # Initialise la matrice des distances

    # Parcourt chaque bloc pour calculer les distances
    for i in range(nb_case):
        for j in range(nb_case):
            # Calcule la divergence L1 entre l'uniformité actuelle et la valeur maximale
            distance = L1(np.array([uniformity[i, j]]), np.array([max_uniformity_value]))
            distances[i, j] = distance

    # Remplace les valeurs NaN dans la matrice des distances par une valeur par défaut
    distances = np.nan_to_num(distances, nan=np.inf)

    # Affichage de la matrice des distances
    affiche_matrice(
        distances,
        "Matrice des distances par rapport à l'uniformité maximale",
        "Distance",
        "viridis_r"
    )

    # Regroupement des distances en classes (10 classes par défaut)
    grouped_distances = group_by_distance_class(distances, num_classes=10)

    # Affichage de la matrice regroupée en classes
    affiche_matrice(
        grouped_distances,
        "Matrice des distances regroupées en 10 classes",
        "Classes de distance",
        "viridis"
    )

    # Affichage des blocs d'image
    affiche_block(Tab_IMG, nb_case)

    # Vérification si l'image a un fond uniforme
    background_status = check_uniform_background(grouped_distances)

    # Affichage du résultat si demandé
    if print_test == 1:
        if background_status == 1:
            print(f"Image {num_image} : Fond Uni (studio)")
        else:
            print(f"Image {num_image} : Pas Fond Uni (report)")

    # Retourne le statut du fond (1 pour uniforme, 0 sinon)
    return background_status


## fin